# Write Images to Bagfile

This notebook writes images from a directory to a new bagfile. Assuming the data directory containt are like so:
> ```
imagedir # images are named using the desired timespamp
    1449692141.10058.png
    1449692141.63769.png
    1449692141.83881.png
    ...
dataDir
    ...
```

The resulting output is like so:

> ```
dataDir
    output.bag # the new bagefile containing the images 
```

In [1]:
import time, sys, os
import rospy
from ros import rosbag
import roslib
roslib.load_manifest('sensor_msgs')
from sensor_msgs.msg import Image

import ImageFile

## Configure parameters

In [1]:
bagfile = 'output.bag' # name of new bagfile
path = '/tmp/bagfiles/' # path for new bagfile
imagedir = '/tmp/imagedir/' # path to image directory
topic_name='/camera/rgb/image_edge' # topic name to publish images under

## Define helper functions

In [2]:
def GetFilesFromDir(dir):
    '''Generates a list of files from the directory'''
    print( "Searching directory %s" % dir )
    all = []
    left_files = []
    right_files = []
    if os.path.exists(dir):
        for path, names, files in os.walk(dir):
            for f in files:
                if os.path.splitext(f)[1] in ['.bmp', '.png', '.jpg']:
                    if 'left' in f or 'left' in path:
                        left_files.append( os.path.join( path, f ) )
                    elif 'right' in f or 'right' in path:
                        right_files.append( os.path.join( path, f ) )
                    all.append( os.path.join( path, f ) )
    return all, left_files, right_files

In [3]:
def CreateMonoBag(imgs, topic, bagname):
    '''Creates a bag file with camera images'''
    bag = rosbag.Bag(bagname, 'w')

    try:
        for i in range(len(imgs)):
#             print("Adding %s" % imgs[i])
            fp = open( imgs[i], "r" )
            p = ImageFile.Parser()

            while 1:
                s = fp.read(1024)
                if not s:
                    break
                p.feed(s)

            im = p.close()
            
            #TODO: change this to read time from filename string
            stamp_sting = os.path.splitext(os.path.basename(imgs[i]))[0]
            Stamp = rospy.Time.from_sec(float(stamp_sting))

            Img = Image()
            Img.header.stamp = Stamp
            Img.width = im.size[0]
            Img.height = im.size[1]
            Img.encoding = "mono8" #for mono images
            #Img.encoding = "rgb8" #for color images
            Img.header.frame_id = "camera_rgb_optical_frame"
            Img_data = [pix for pix in im.getdata()] #for mono images
            #Img_data = [pix for pixdata in im.getdata() for pix in pixdata] #for color images
            Img.data = Img_data
s
            bag.write(topic, Img, Stamp)
    finally:
        bag.close()

In [4]:
def CreateBag(imagedir, topic, bagname):
    '''Creates the actual bag file by successively adding images'''
    all_imgs, left_imgs, right_imgs = GetFilesFromDir(imagedir)
    if len(all_imgs) <= 0:
        print("No images found in %s" % args[0])
        exit()

    CreateMonoBag(all_imgs, topic, bagname)

## Create bagfile from images

In [5]:
CreateBag(imagedir, topic_name, os.path.join(path,bagfile))

Searching directory /home/ruffsl/Downloads/lol/edge_images


## Merge new bagfile with another
To merge our new output bagfile with another, to that we can get just one bagfile that containes all topics, we can use a python script in a small ros package found here:
https://bitbucket.org/daniel_dube/bagedit/wiki/Home

After you've added the package to your ros path, the command will look something like this

`rosrun bagedit bagmerge.py -o final.output.bag -i output.bag another.bag`